## foo

In [26]:
from IPython.display import JSON

In [11]:
# sentence embeddings
from sentence_transformers import SentenceTransformer

# now we need a model to generate embeddings
sentence_embedding_model_name = "all-MiniLM-L6-v2"
sentence_embedding_model = SentenceTransformer(sentence_embedding_model_name)

reviews = [
    "I love the food in this app.",
    "The service was terrible.",
    "Great experience overall!",
    "I would not recommend this app to anyone.",
    "The food was delicious but the wait time was too long."
]

review_embeddings = sentence_embedding_model.encode(reviews)

for i, sentence in enumerate(reviews):
    print(f"Sentence: {sentence}")
    print(f"Embedding: {review_embeddings[i]}")
    print()

Sentence: I love the food in this app.
Embedding: [-4.63027358e-02 -9.96123906e-03  2.34699361e-02  3.51960473e-02
 -3.30975279e-02 -9.20271315e-03  4.81258743e-02 -2.64003184e-02
  2.31640674e-02 -2.51029544e-02  1.29931606e-02 -4.55674306e-02
 -2.36642249e-02 -4.95195724e-02  4.49798815e-02 -6.01701103e-02
  1.59362674e-01 -4.10257466e-02 -3.74425426e-02 -7.29371756e-02
 -4.20963950e-02  2.31624041e-02  7.00441003e-02  1.51903797e-02
  2.40844227e-02 -2.17394270e-02 -1.60519816e-02 -3.82816829e-02
  7.88023230e-03 -4.08013724e-02  4.94900644e-02  2.84704962e-03
  2.72149332e-02  2.59417929e-02 -1.04830958e-01 -5.64206094e-02
  7.82919452e-02 -1.11386195e-01 -1.03928838e-02 -4.92460607e-03
  3.13421860e-02 -4.27873246e-03  9.36537702e-03  2.92871296e-02
 -9.93652269e-03 -4.32859361e-02 -7.65798613e-02 -1.11036655e-02
  1.16450451e-01  6.92716315e-02 -1.35540375e-02 -2.47192495e-02
  2.39385255e-02  6.21156394e-03  2.41133329e-02  5.54640405e-02
  1.36228688e-02 -4.04334180e-02  1.0450

In [3]:
review_embeddings[0].shape

(384,)

In [2]:
from qdrant_client import QdrantClient

In [3]:
client = QdrantClient(
    host = 'localhost',
    port = 6333
)

In [4]:
collectionResponse = client.get_collections()

In [11]:
collectionResponse

CollectionsResponse(collections=[CollectionDescription(name='reviews')])

In [19]:
client.delete_collection(
    collection_name = "reviews"
)

True

In [20]:
client.create_collection(
    collection_name = "reviews",
    vectors_config = {
            "size": 384,
            "distance": "Cosine"
    }
)

True

In [6]:
collection = client.get_collection(collection_name="reviews")

In [27]:
JSON(collection.config.model_dump_json())

<IPython.core.display.JSON object>

In [21]:
points = [{
    "id": i,
    "vector": review_embedding,
    "payload": {
        "review": reviews[i]
    }
} for i,review_embedding in enumerate(review_embeddings)]

In [22]:
client.upsert(
    collection_name = "reviews",
    points = points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [26]:
client.query_points(
    collection_name = "reviews",
    query = review_embeddings[0],
    limit=5
)

QueryResponse(points=[ScoredPoint(id=0, version=0, score=1.0, payload={'review': 'I love the food in this app.'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=3, version=0, score=0.45042762, payload={'review': 'I would not recommend this app to anyone.'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=4, version=0, score=0.41885424, payload={'review': 'The food was delicious but the wait time was too long.'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=2, version=0, score=0.2064805, payload={'review': 'Great experience overall!'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=1, version=0, score=0.1179505, payload={'review': 'The service was terrible.'}, vector=None, shard_key=None, order_value=None)])

In [34]:
client.query_points(
    collection_name = "reviews",
    limit=5,
    query = sentence_embedding_model.encode("I like this food")
)

QueryResponse(points=[ScoredPoint(id=0, version=0, score=0.6067921, payload={'review': 'I love the food in this app.'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=4, version=0, score=0.36157715, payload={'review': 'The food was delicious but the wait time was too long.'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=3, version=0, score=0.14748323, payload={'review': 'I would not recommend this app to anyone.'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=2, version=0, score=0.14049788, payload={'review': 'Great experience overall!'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=1, version=0, score=0.019978367, payload={'review': 'The service was terrible.'}, vector=None, shard_key=None, order_value=None)])